# Calculate $\omega \text{s}$ and corresponding eigenvectors

In [218]:
from mlat.const import dirac, pauli
import numpy as np
import numpy.linalg as la

u = 5.
N = 10

d = dirac(N, N)
p = pauli()

H = 0

for mx in range(1, N-2):
    for my in range(1, N-1):
        fst =  np.kron(d.ket(mx+1, my).dot(d.bra(mx, my)), (p.z + 1.j * p.x) / 2.)
        H += fst + fst.conj().T
for mx in range(1, N-1):
    for my in range(1, N-2):
        fst =  np.kron(d.ket(mx, my+1).dot(d.bra(mx, my)), (p.z + 1.j * p.y) / 2.)
        H += fst + fst.conj().T
for mx in range(1, N-1):
    for my in range(1, N-1):
        fst =  np.kron(d.ket(mx, my).dot(d.bra(mx, my)), p.z)
        H += u * fst

M = np.diag([m] * H.shape[0])


evals, evecs = la.eig(la.inv(M).dot(H))
idcs = np.argsort(evals)
evals, evecs = evals[idcs], evecs[idcs]

ws = np.sqrt(np.abs(evals.real))

In [187]:
# print(ws)
print(np.argmin(np.abs(ws-1)))
# print(np.where(ws 3.27458412e-01))
print(evecs.shape)
print(ws)
# ws[500]

63
(200, 200)
[2.62545984 2.59566161 2.59560763 2.56564288 2.54813349 2.54813176
 2.51797737 2.5177091  2.48655858 2.48652217 2.46951944 2.45580708
 2.4557918  2.41637453 2.41637152 2.40700797 2.40634565 2.38509307
 2.38496136 2.34535093 2.34533051 2.34267606 2.33484098 2.33476592
 2.3132771  2.31326172 2.28313676 2.28313473 2.26973318 2.26861976
 2.26185748 2.26163338 2.25034907 2.25030328 2.24013395 2.24013091
 2.20676121 2.20676052 2.19757403 2.19753398 2.1946844  2.19406312
 2.19283253 2.15292451 2.1528915  2.12797732 2.12772704 2.11636015
 2.11499144 2.08165348 2.08164699 2.04662675 2.0466015  2.03454374
 1.99824003 1.99811629 1.96248733 1.9614005  1.91070258 1.91069577
 1.8854682  1.83130311 1.83092517 1.7741737  0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.       

# Animation

In [219]:
import numpy as np
import plotly.graph_objects as go

# N=10 -78
mode = 20
w = ws[mode] 

grid_cell_size = 2

grid = np.array([[[grid_cell_size * i, grid_cell_size * j] for i in range(N)] for j in range(N)], dtype=np.complex128)

# Print infos
print("Angular frequency: ", w)

# Parameters for animation
fps = 30 # /s
animation_length = 10 # s

dgrid = np.zeros(grid.shape, dtype=np.complex128)

for mx in range(N):
    for my in range(N):
        dgrid[my, mx] = evecs[mode].dot(np.kron(d.ket(mx, my), np.eye(2)))
#         dgrid[my, mx] = evecs[mode].dot(np.kron(np.eye(2), d.ket(mx, my)))
        



# Construct frames
frames = []
for t in range(int(animation_length * fps)):
    dt = t / fps
    dphase = dt * w * 2 * np.pi
    current_grid = grid + dgrid * np.exp(1.j * dphase)
    x = current_grid[:, :, 0].flatten()
    y = current_grid[:, :, 1].flatten()
    frames.append(go.Frame(data=[go.Scatter(x=x.real, y=y.real, mode='markers')]))

# Figure components
start_button = dict(
        label="Play",
        method="animate",
        args=[
            None, 
            {
                "frame":{"duration": 1000 / fps, "redraw": False},
                "fromcurrent": True, 
                "transition": {"duration": 100}
            }])
pause_button = dict(
        label="Pause",
        method="animate",
        args=[
            [None], 
            {
                "frame": {"duration": 0, "redraw": False},
                "mode": "immediate",
                "transition": {"duration": 0}
            }])

# Plot
fig = go.Figure(
    data=frames[0].data,
    layout=go.Layout(
        title="Dispersion relation animation",
        yaxis=dict(scaleanchor="x", scaleratio=1, autorange='reversed'),
        updatemenus=[
            dict(
                type="buttons",
                buttons=[start_button, pause_button
            ])
        ]
    ),
    frames=frames[1:])
fig.show()

Angular frequency:  2.345330511994498
